In [1]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox

In [2]:
# --- SMTP Configuration ---
SENDER_EMAIL = "fahadbokaro@gmail.com"
SENDER_PASSWORD = "lnsn wifh hlcg urqq"  
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

In [3]:
PRIMARY_KEY = 'SAIL_PERNO'
MASTER_PATH = "master_file.csv"
CHANGES_PATH = "changes_file.csv"

def read_csv(file_path):
    return pd.read_csv(file_path, dtype=str)


In [4]:
def show_table(df, frame):
    for widget in frame.winfo_children():
        widget.destroy()

    tree = ttk.Treeview(frame, show="headings")

    columns = ["Index"] + list(df.columns)
    tree["columns"] = columns

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=120, anchor='w')

    for i, (_, row) in enumerate(df.iterrows(), start=1):
        values = [i] + list(row)
        tree.insert("", "end", values=values)

    tree_scroll = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=tree_scroll.set)

    tree.grid(row=0, column=0, sticky="nsew")
    tree_scroll.grid(row=0, column=1, sticky="ns")
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)


In [5]:
def show_flat_grouped_table(df, frame):
    for widget in frame.winfo_children():
        widget.destroy()

    tree = ttk.Treeview(frame, show="headings")

    columns = ["Index"] + list(df.columns)
    tree["columns"] = columns

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=90, anchor='w')

    df_sorted = df.sort_values(by=PRIMARY_KEY)

    previous_key = None
    index_counter = 1

    for _, row in df_sorted.iterrows():
        current_key = row[PRIMARY_KEY]

        # Insert blank line between different employees
        if previous_key is not None and current_key != previous_key:
            tree.insert("", "end", values=[""] * len(columns))

        # Check if this is the first row for a new employee
        if current_key != previous_key:
            index_display = index_counter
            index_counter += 1
        else:
            index_display = ""

        values = [index_display] + list(row)
        tree.insert("", "end", values=values)

        previous_key = current_key

    # Scrollbar setup
    tree_scroll = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=tree_scroll.set)

    tree.grid(row=0, column=0, sticky="nsew")
    tree_scroll.grid(row=0, column=1, sticky="ns")
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)


In [6]:
def compare_data(master_df, changes_df):
    results = []

    master_df.set_index(PRIMARY_KEY, inplace=True)
    changes_df.set_index(PRIMARY_KEY, inplace=True)

    common_keys = master_df.index.intersection(changes_df.index)

    for key in common_keys:
        master_row = master_df.loc[key]
        changes_row = changes_df.loc[key]

        name = master_row["NAME"] if "NAME" in master_row else "---"

        common_columns = master_row.index.intersection(changes_row.index)
        for col in common_columns:
            old = master_row[col]
            new = changes_row[col]

            if pd.isna(old) and pd.isna(new):
                continue
            if old != new:
                results.append([key, name, col, old, new])

    comparison_df = pd.DataFrame(results, columns=[PRIMARY_KEY, "NAME", "Column Name", "Old Value", "New Value"])
    return comparison_df, len(results)


In [7]:
def get_new_employees(master_df, changes_df):
    master_keys = set(master_df['SAIL_PERNO'].astype(str))
    changes_df['SAIL_PERNO'] = changes_df['SAIL_PERNO'].astype(str)

    new_employees = changes_df[~changes_df['SAIL_PERNO'].isin(master_keys)]

    # Desired column order
    cols_to_show = ['NAME', 'SAIL_PERNO', 'DOJ_SAIL', 'DOB', 'PAN', 'IFSC_CD']
    available_cols = [col for col in cols_to_show if col in new_employees.columns]

    return new_employees[available_cols]


In [8]:
def clean_dataframe(df):
    # PRAN_NO: If numeric, convert to int (or string without decimal point)
    if 'PRAN_NO' in df.columns:
        df['PRAN_NO'] = df['PRAN_NO'].apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else x)
        
    if 'BANK_ACN' in df.columns:
        df['BANK_ACN'] = df['BANK_ACN'].apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else x)
    
    if 'Old Value' in df.columns:
        df['Old Value'] = df['Old Value'].apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else x)
        
    if 'New Value' in df.columns:
        df['New Value'] = df['New Value'].apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else x)
    
    # Replace NaN with blank string for all cells
    df = df.fillna('---')
    
    return df


In [9]:
import pdfkit
import tempfile

# Path to wkhtmltopdf — change if installed elsewhere
config = pdfkit.configuration(wkhtmltopdf="C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe")

def save_df_as_pdf(df, filename, heading="Data Report"):
    """Convert a DataFrame to a styled PDF with optional heading using HTML and pdfkit."""
    
    # Define custom CSS style for PDF
    styles = """
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            margin: 40px;
            color: #2c3e50;
        }
        h1 {
            text-align: center;
            margin-bottom: 30px;
            color: #007bff;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            font-size: 14px;
        }
        th, td {
            border: 1px solid #ccc;
            padding: 10px 15px;
            text-align: center;
        }
        th {
            background-color: #f0f4f8;
        }
        tr:nth-child(even) {
            background-color: #f9fcff;
        }
    </style>
    """

    # Convert DataFrame to HTML table
    html_table = df.to_html(index=False, border=0, justify='center')

    # Combine all into full HTML
    html = f"""
    <html>
    <head>{styles}</head>
    <body>
        <h1>{heading}</h1>
        {html_table}
    </body>
    </html>
    """

    with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as temp_html:
        temp_html.write(html.encode('utf-8'))
        temp_html_path = temp_html.name

    try:
        pdfkit.from_file(temp_html_path, filename, configuration=config)
        print(f"✅ Saved PDF to {filename}")
    except Exception as e:
        print("❌ PDF generation failed:", e)


In [10]:
from tkinter import filedialog

def open_email_window():
    email_win = tk.Toplevel()
    email_win.title("Send Report via Email")
    email_win.geometry("400x300")
    email_win.configure(bg="#f5f7fa")

    tk.Label(email_win, text="Recipient Email:", font=("Segoe UI", 11), bg="#f5f7fa").pack(pady=10)
    email_entry = tk.Entry(email_win, width=40)
    email_entry.pack()

    file_paths = []

    def choose_files():
        files = filedialog.askopenfilenames(filetypes=[("PDF Files", "*.pdf")])
        file_paths.extend(files)
        file_label.config(text=f"{len(file_paths)} file(s) selected")

    tk.Button(email_win, text="Choose Files", command=choose_files).pack(pady=10)
    file_label = tk.Label(email_win, text="No files selected", bg="#f5f7fa")
    file_label.pack()

    def send_files():
        recipient = email_entry.get()
        if not recipient or not file_paths:
            messagebox.showerror("Error", "Please provide an email and select at least one file.")
            return
        send_email_with_attachments(recipient, file_paths)
        email_win.destroy()

    tk.Button(email_win, text="Send Email", command=send_files, bg="#007bff", fg="white").pack(pady=20)


In [11]:
import smtplib
from email.message import EmailMessage
import os

def send_email_with_attachments(recipient_email, file_paths):
    msg = EmailMessage()
    msg["Subject"] = "Requested Employee Report(s)"
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content("Please find the attached report(s).")

    # Attach files
    for path in file_paths:
        with open(path, "rb") as f:
            file_data = f.read()
            file_name = os.path.basename(path)
            msg.add_attachment(file_data, maintype="application", subtype="octet-stream", filename=file_name)

    # Send email
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)
            messagebox.showinfo("Success", "Email sent successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to send email: {e}")


In [12]:
def launch_gui():
    master_df = pd.read_csv(MASTER_PATH)
    changes_df = pd.read_csv(CHANGES_PATH)

    master_df = clean_dataframe(master_df)
    changes_df = clean_dataframe(changes_df)

    root = tk.Tk()
    root.title("Employee Data Comparison Tool")
    root.geometry("1400x850")
    root.configure(bg="#f0f2f5")

    # ---------- Styling ----------
    style = ttk.Style()
    style.theme_use("default")

    # Define Colors and Fonts
    COLOR_PRIMARY = "#007bff"
    COLOR_SECONDARY = "#6c757d"
    COLOR_SUCCESS = "#28a745"
    COLOR_WARNING = "#fd7e14"
    COLOR_PURPLE = "#6f42c1"
    COLOR_BG = "#f0f2f5"

    # Bigger fonts
    FONT_HEADER = ("Segoe UI", 20, "bold")
    FONT_BTN = ("Segoe UI", 10)
    FONT_TABLE = ("Segoe UI", 10)

    # Configure Button Styles
    style.configure("Primary.TButton", font=FONT_BTN, foreground="white", background=COLOR_PRIMARY, padding=10)
    style.map("Primary.TButton", background=[("active", "#0056b3")])

    style.configure("Secondary.TButton", font=FONT_BTN, foreground="white", background=COLOR_SECONDARY, padding=10)
    style.map("Secondary.TButton", background=[("active", "#5a6268")])

    style.configure("Success.TButton", font=FONT_BTN, foreground="white", background=COLOR_SUCCESS, padding=10)
    style.map("Success.TButton", background=[("active", "#1e7e34")])

    style.configure("Warning.TButton", font=FONT_BTN, foreground="white", background=COLOR_WARNING, padding=10)
    style.map("Warning.TButton", background=[("active", "#e8590c")])

    style.configure("Email.TButton", font=FONT_BTN, foreground="white", background=COLOR_PURPLE, padding=10)
    style.map("Email.TButton", background=[("active", "#5a32a3")])

    # ---------- Title ----------
    title_lbl = ttk.Label(root, text="📊 Employee Data Comparison Tool", font=FONT_HEADER, background=COLOR_BG, foreground="#343a40")
    title_lbl.pack(pady=(30, 10))

    underline = tk.Frame(root, bg=COLOR_PRIMARY, height=4, width=500)
    underline.pack(pady=(0, 30))

    # ---------- Button Frame ----------
    button_frame = ttk.Frame(root, padding=20)
    button_frame.pack()

    def show_master():
        show_table(master_df, table_frame)

    def show_changes():
        show_table(changes_df, table_frame)

    def show_comparison():
        comparison_df, total_changes = compare_data(master_df.copy(), changes_df.copy())
        comparison_df = clean_dataframe(comparison_df)
        if comparison_df.empty:
            messagebox.showinfo("No Changes", "No differences found.")
        else:
            show_flat_grouped_table(comparison_df, table_frame)

    def get_new_employees_button_clicked():
        try:
            m_df = read_csv(MASTER_PATH)
            c_df = read_csv(CHANGES_PATH)
            new_employees_df = get_new_employees(m_df, c_df)
            new_employees_df = clean_dataframe(new_employees_df)
            show_table(new_employees_df, table_frame)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def save_changes_pdf():
        try:
            comparison_df, total_changes = compare_data(master_df.copy(), changes_df.copy())
            comparison_df = clean_dataframe(comparison_df)
            if not comparison_df.empty:
                save_df_as_pdf(comparison_df, "changes.pdf", heading="Employee Data Changes Report")
                messagebox.showinfo("Success", "Comparison saved as PDF")
            else:
                messagebox.showinfo("No Changes", "No differences found to save.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def save_new_employees_pdf():
        try:
            m_df = read_csv(MASTER_PATH)
            c_df = read_csv(CHANGES_PATH)
            new_employees_df = get_new_employees(m_df, c_df)
            new_employees_df = clean_dataframe(new_employees_df)
            save_df_as_pdf(new_employees_df, "new_employees.pdf", heading="New Employees Joined")
            messagebox.showinfo("Success", "New Employees saved as PDF")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    # ---------- Buttons Row 1 ----------
    ttk.Button(button_frame, text="📂 Master File", command=show_master, style="Secondary.TButton").grid(row=0, column=0, padx=10, pady=10)
    ttk.Button(button_frame, text="📂 Changes File", command=show_changes, style="Secondary.TButton").grid(row=0, column=1, padx=10, pady=10)
    ttk.Button(button_frame, text="🔍 Get Changes", command=show_comparison, style="Primary.TButton").grid(row=0, column=2, padx=10, pady=10)
    ttk.Button(button_frame, text="🧾 New Employees", command=get_new_employees_button_clicked, style="Success.TButton").grid(row=0, column=3, padx=10, pady=10)

    # ---------- Buttons Row 2 (Export) ----------
    export_frame = ttk.Frame(root, padding=20)
    export_frame.pack()

    ttk.Button(export_frame, text="📤 Save Changes PDF", command=save_changes_pdf, style="Warning.TButton").grid(row=0, column=0, padx=10, pady=10)
    ttk.Button(export_frame, text="📤 Save New Employees PDF", command=save_new_employees_pdf, style="Warning.TButton").grid(row=0, column=1, padx=10, pady=10)
    ttk.Button(export_frame, text="📧 Email Reports", command=open_email_window, style="Email.TButton").grid(row=0, column=2, padx=10, pady=10)

    # ---------- Table Display Frame ----------
    table_frame = ttk.Frame(root, relief="sunken", padding=20)
    table_frame.pack(fill="both", expand=True, padx=30, pady=30)

    show_table(master_df, table_frame)

    root.mainloop()


In [13]:
launch_gui()